In [ ]:
import csv
from matplotlib import pyplot as plt
from matplotlib import cm
import numpy as np
from scipy.interpolate import make_interp_spline, BSpline, make_lsq_spline
from scipy.optimize import curve_fit
from curlyBrace import curlyBrace

# Colormap: paired
# Extract colors from colormap, to use at will
colormap = cm.get_cmap('Paired')
colors = [colormap(i) for i in np.linspace(0, 1, 12)]

# Define fitting function
def fitting_exponential(x, y0, v0, k):
    return y0 - (v0/k) * (1 - np.exp(-k*x))

def fitting_sigmoid(x, a, b, c, d):
    return d + (a - d)/(1 + np.power(x / c, b))

# Wrappers for easier use of interpolate functions from scipy
def generate_interp_spline(time, product, time_smooth):
    time = time[:len(product)]
    spl = make_interp_spline(time, product, k=3)  # type: BSpline
    smooth_data = spl(time_smooth)
    return smooth_data

def generate_lsq_spline(time, product, time_smooth):
    time = time[:len(product)]
    popt, _ = curve_fit(fitting_exponential, time, product)
    smooth_data = fitting_exponential(time_smooth, *popt)
    return smooth_data

In [ ]:
# --- Setup data for AA carbonylation kinetics ---

SA_time = list()
SA_yield = dict([("dcpe-0.5M", []), ("dppe-1M", []), ("dcpe-1M", []), ("PPh3-1M", []), ("noligand-1M", [])])

with open('data/AA_Carbonylation_Figure1.csv', 'r') as data_file:
    plots = csv.reader(data_file, delimiter=';')
    next(plots)
    for row in plots:
        SA_time.append(row[0])
        SA_yield['dcpe-0.5M'].append(row[1])
        SA_yield['dppe-1M'].append(row[2])
        SA_yield['dcpe-1M'].append(row[3])
        SA_yield['PPh3-1M'].append(row[4])
        SA_yield['noligand-1M'].append(row[5])

# Cast str to float
SA_time = [float(i) for i in SA_time]
SA_yield = {j:[float(i) for i in SA_yield[j] if i] for j in SA_yield}

# Create smooth lines on data
# 300 represents number of points to make between T.min and T.max
time_smooth = np.linspace(0, 48, 300)
smooth_data = dict([("dcpe-0.5M", []), ("dppe-1M", []), ("dcpe-1M", []), ("PPh3-1M", []), ("noligand-1M", [])])

# Spline interpolation
# smooth_data["dcpe-0.5M"] = generate_interp_spline(SA_time, SA_yield["dcpe-0.5M"], time_smooth)

# Exponential fitting
smooth_data["dcpe-0.5M"] = generate_lsq_spline(SA_time, SA_yield["dcpe-0.5M"], time_smooth)

# Testing lsq data:
# knots = np.linspace(FA_time['Co-3'][0], FA_time['Co-3'][-1], 3)
# knots_t = np.r_[(FA_time['Co-3'][0],)*3, knots, (FA_time['Co-3'][-1],)*3]
# spl = make_lsq_spline(FA_time['Co-3'], FA_yield['Co-3'], t=knots_t, k=3)  # type: BSpline
# smooth_data["Co-3"] = spl(time_smooth)

# --- Starting plotting --- #
# Comparison of catalysts
axis = plt.gca()
axis.set_axisbelow(True)
axis.set_xlabel("Time (h)")
axis.set_xlim(0, 48)
axis.set_ylim(0, 100)
axis.set_ylabel("Reaction yield ($\%$)")

# If secondary axis is required (for yield for instance)
# axis2 = axis.twinx()
# axis2.set_ylabel("Yield ($\%$)")
# axis2.set_ylim(-5, 100)

# Plot each dataset plus smoothing with selected colors.
# Zorder value is used to put it in front of other lines in the graph
plt.plot(SA_time[:len(SA_yield['dcpe-0.5M'])], SA_yield['dcpe-0.5M'],'o', zorder=2.6, color=colors[6])
plt.plot(time_smooth, smooth_data['dcpe-0.5M'], '-', zorder=2.8, label="1", color=colors[7])
plt.legend(fontsize='small')

plt.tight_layout()
plt.savefig("../data/AA_Carbonylation.png", dpi=300)